In [2]:
import numpy as np

In [3]:
data = open('data/day12.txt', 'r').read()

data_arr = np.zeros((140, 140)).astype(str)
for index, line in enumerate(data.split('\n')[:-1]):
    data_arr[index] = list(line)

data_arr

array([['U', 'U', 'U', ..., 'R', 'R', 'R'],
       ['U', 'U', 'U', ..., 'R', 'R', 'R'],
       ['U', 'U', 'U', ..., 'B', 'R', 'R'],
       ...,
       ['R', 'R', 'R', ..., 'J', 'O', 'O'],
       ['R', 'R', 'R', ..., 'J', 'O', 'O'],
       ['R', 'R', 'R', ..., 'J', 'O', 'O']], dtype='<U32')

In [4]:
data_test = open('other/day12_test.txt', 'r').read()

data_test_arr = np.zeros((10, 10)).astype(str)
for index, line in enumerate(data_test.split('\n')[:-1]):
    data_test_arr[index] = list(line)

data_test_arr

array([['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'F', 'F'],
       ['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'C', 'F'],
       ['V', 'V', 'R', 'R', 'R', 'C', 'C', 'F', 'F', 'F'],
       ['V', 'V', 'R', 'C', 'C', 'C', 'J', 'F', 'F', 'F'],
       ['V', 'V', 'V', 'V', 'C', 'J', 'J', 'C', 'F', 'E'],
       ['V', 'V', 'I', 'V', 'C', 'C', 'J', 'J', 'E', 'E'],
       ['V', 'V', 'I', 'I', 'I', 'C', 'J', 'J', 'E', 'E'],
       ['M', 'I', 'I', 'I', 'I', 'I', 'J', 'J', 'E', 'E'],
       ['M', 'I', 'I', 'I', 'S', 'I', 'J', 'E', 'E', 'E'],
       ['M', 'M', 'M', 'I', 'S', 'S', 'J', 'E', 'E', 'E']], dtype='<U32')

In [264]:
data2_test = open('other/day12_test2.txt', 'r').read()

data2_test_arr = np.zeros((6, 6)).astype(str)
for index, line in enumerate(data2_test.split('\n')[:-1]):
    data2_test_arr[index] = list(line)

data2_test_arr

array([['A', 'A', 'A', 'A', 'A', 'A'],
       ['A', 'A', 'A', 'B', 'B', 'A'],
       ['A', 'A', 'A', 'B', 'B', 'A'],
       ['A', 'B', 'B', 'A', 'A', 'A'],
       ['A', 'B', 'B', 'A', 'A', 'A'],
       ['A', 'A', 'A', 'A', 'A', 'A']], dtype='<U32')

In [274]:
class Field():
    def __init__(self, data):
        self.field_init = data.copy()
        self.field = data.copy()
        self.size = len(self.field)
        self.vege = np.unique(self.field)
        self.vege_new = self.vege

    def get_neighbours(self, x, y):
        result = []
        if x != 0:
            result.append((x-1, y))
        if x != self.size-1:
            result.append((x+1, y))
        if y != 0:
            result.append((x, y-1))
        if y != self.size-1:
            result.append((x, y+1))
        return result

    def nb_split(self, x, y):
        return 4-sum([1 for neigh in self.get_neighbours(x, y) if self.field[neigh] == self.field[x][y]])

    def change_part(self, x, y, old_value, new_value):
        for neigh in self.get_neighbours(x, y):
            if self.field[neigh] == old_value:
                self.field[neigh] = new_value
                self.change_part(neigh[0], neigh[1], old_value, new_value)
        return self

    def change_field(self):
        for lin in range(self.size):
            for col in range(self.size):
                if self.field[lin][col][0] != '.':
                    new_val = f'.{lin}-{col}'
                    self.change_part(lin, col, self.field[lin][col],new_val)
                    self.field[lin][col] = new_val
        self.vege_new = np.unique(self.field)
        return self

    def price_field(self):
        self.change_field()
        counts = np.unique(self.field, return_counts=True)
        result = {counts[0][index]:[counts[1][index], 0] for index in range(len(counts[0]))}
        for lin in range(self.size):
            for col in range(self.size):
                result[self.field[lin][col]][1] += self.nb_split(lin, col)
        price = 0
        for elem in result.values():
            price += elem[0]*elem[1]
        return price

    ## Part 2
    def get_parcel(self, type_vege):
        xs, ys = np.where(self.field == type_vege)
        min_x, max_x = np.min(xs), np.max(xs)
        min_y, max_y = np.min(ys), np.max(ys)
        return self.field[min_x:max_x+1, min_y: max_y+1]

    def expand_parcel(self, parcel):
        height, width = len(parcel), len(parcel[0])
        exp_parcel = []
        for lin in range(2*height+1):
            if lin % 2 == 0:
                exp_parcel.append(['.']*(2*width+1))
            else:
                line = ['.']
                for col in range(width):
                    line.append(parcel[(lin-1)//2, col])
                    line.append('.')
                exp_parcel.append(line)
        return np.array(exp_parcel)

    def draw_bounds(self, exp_parcel, type_vege):
        height, width = len(exp_parcel), len(exp_parcel[0])
        for lin in range(height):
            for col in range(width):

                vert_n, horiz_n = [], []
                if lin != 0:
                    vert_n.append((lin-1, col))
                if lin != height-1:
                    vert_n.append((lin+1, col))
                if col != 0:
                    horiz_n.append((lin, col-1))
                if col != width-1:
                    horiz_n.append((lin, col+1))

                if len(vert_n) == 1:
                    if exp_parcel[vert_n[0]] == type_vege:
                        exp_parcel[lin, col] = '----'
                else:
                    if (exp_parcel[vert_n[0]] == type_vege and exp_parcel[vert_n[1]] != type_vege) or (exp_parcel[vert_n[0]] != type_vege and exp_parcel[vert_n[1]] == type_vege):
                        exp_parcel[lin, col] = '----'

                if len(horiz_n) == 1:
                    if exp_parcel[horiz_n[0]] == type_vege:
                        exp_parcel[lin, col] = '|'
                else:
                    if (exp_parcel[horiz_n[0]] == type_vege and exp_parcel[horiz_n[1]] != type_vege) or (exp_parcel[horiz_n[0]] != type_vege and exp_parcel[horiz_n[1]] == type_vege):
                        exp_parcel[lin, col] = '|'

        return exp_parcel

    def count_angles(self, bounded_parcel):
        nb_angles = 0
        height, width = len(bounded_parcel), len(bounded_parcel[0])
        for lin in range(height):
            for col in range(width):

                neighs = []
                if lin != 0:
                    if bounded_parcel[lin-1, col] == '|':
                        neighs.append(1)
                if lin != height-1:
                    if bounded_parcel[lin+1, col] == '|':
                        neighs.append(1)
                if col != 0:
                    if bounded_parcel[lin, col-1] == '----':
                        neighs.append(2)
                if col != width-1:
                    if bounded_parcel[lin, col+1] == '----':
                        neighs.append(2)

                neighs.sort()
                if neighs == [1, 2]:
                    nb_angles += 1
                if neighs == [1, 1, 2, 2]:
                    nb_angles += 2


        return nb_angles

    def price_field_2(self):
        self.change_field()
        counts = np.unique(self.field, return_counts=True)
        result = {counts[0][index]:[counts[1][index], 0] for index in range(len(counts[0]))}
        for vege in self.vege_new:
            result[vege][1] = self.count_angles(self.draw_bounds(self.expand_parcel(self.get_parcel(vege)), vege))

        price = 0
        for elem in result.values():
            price += elem[0]*elem[1]
        return price

In [275]:
field = Field(data_arr)

In [276]:
field.price_field()

1370100

In [277]:
field.price_field_2()

818286